# 키워드 분석

In [1]:
import matplotlib.font_manager as fm

sys_font = fm.findSystemFonts()

[f for f in sys_font if 'Nanum' in f]

['C:\\Users\\koreacamel\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumBarunGothic.ttf',
 'C:\\Users\\koreacamel\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumGothicExtraBold.ttf',
 'C:\\Users\\koreacamel\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumBarunGothicLight.otf',
 'C:\\Users\\koreacamel\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumBarunGothicLight.ttf',
 'C:\\Users\\koreacamel\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumBarunGothicUltraLight.otf',
 'C:\\Users\\koreacamel\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumBarunGothicUltraLight.ttf',
 'C:\\Users\\koreacamel\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumBarunGothicBold.ttf',
 'C:\\Users\\koreacamel\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumBarunGothic.otf',
 'C:\\Users\\koreacamel\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumBarunGothicBold.otf']

In [2]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

article_df= pd.read_csv("data/article_2000.csv")


C:\Users\koreacamel\AppData\Local\Temp\ipykernel_15736\312527451.py:4: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')


In [4]:
# 필요한 컬럼 리스트만들기

#언론사 이름  - 불용어 처리를 위해서
press_list = list(article_df.press)

#기사 본문 
articles_main = list(article_df.Article)

#기사 제목
articles_title = list(article_df.Title)

### 전처리 
- 불용어 사전 만들기

In [ ]:
#불용어 사전 가지고 오기 
f = open("data/koreanStopwords.txt", 'r', encoding = 'utf-8')
lines = f.readlines()
stopwords_korean = []
for line in lines:
    line = line.replace('\n', '')
    stopwords_korean.append(line)
f.close()

#결과물 확인하며 반복으로 추가 한다. 
stop_add1 = '삼성전자 베스트 수량 추천 쿠폰 파격 최초 공개 삼성 갤럭시 구매 플랫 폼 네이버 시 뉴스 진행 롯데 티몬 지난해 이번 쇼 세트 카카오 오후 관계자 플 번가 제품 소개 브랜드 현대홈쇼핑 배 온스타일 닷컴 공영 '
stop_add1 = stop_add1.split(' ')

stop_add2 = '특가 할인 세일 역대 타임 한정 시즌 사은품 무배 마지막 데일리 준비 인기 최저 기념 기획 코코 베베 위크 단독 추가 필수 신상 최대 특집 혜택 매트 증정 컬러 이벤트 데이 특별 무료 본사 모음 출시 프리미엄'
stop_add2 = stop_add2.split(' ')


stop_add3 = '라방 라이브 방송 전 커머스 쇼핑 상품 등 고객 판매 홈 난 일 걸 뭐 줄 만 건 분 개 끝 잼 이거 번 중 듯 때 게 내 말 나 수 거 점 것 제일제당 기사 기자 라이브'
stop_add3 = stop_add3.split(' ')


stop_words = stopwords_korean + stop_add1 +stop_add2 + stop_add3

- 기사 본문, 제목에서 분석에 필요없는 단어 제거하기 위해서 정규식으로 한글만 남긴다. 

In [5]:
import re

articles_main_clean = [ ]

for article in articles_main:
  clean = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", " ", article)
  articles_main_clean.append(clean)

articles_title_clean = [ ]

for title in articles_title:
  clean = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", " ", title)
  articles_title_clean.append(clean)


### 데이터 추출 
- 텍스트 분석을 위해 KoNLPy(“코엔엘파이”)는 한국어 정보처리를 위한 파이썬 패키지활용
- Mecab 으로 형태소 분석하여 길이가 1이상인 일반명사, 고유명사, 형용사, 어근만 추출한다. 
- 불용어사전을 통해서 전처리 

In [9]:
from konlpy.tag import Mecab

tagger = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

nouns_article = [ ]
for article in articles_main_clean:
  tagged = tagger.pos(article)
  nouns = [s for s, t in tagged if t in ['NNG', 'NNP', 'VA', 'XR'] and len(s) >1]
  for noun in nouns:
    if noun not in stop_words:
        nouns_article.append(noun)

- 파이썬 Counter 패키지를 통해 추출한 단어의 빈도수를 측정한다. 

In [10]:
from collections import Counter

nouns_counter = Counter(nouns_article)

#상위 200개 데이터
top_nouns_article = dict(nouns_counter.most_common(200))


### 데이터 시각화

- 빈도수 상위 200개 데이터로 워드클라우드 생성 

In [11]:
from wordcloud import WordCloud

wc = WordCloud(background_color = 'white', font_path = 'C:\\Users\\koreacamel\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumBarunGothicBold.otf')

wc.generate_from_frequencies(top_nouns_article) 
figure =  plt.figure(figsize = (30, 30))
ax =  figure.add_subplot(1, 1, 1)
ax.axis('off')
ax.imshow(wc)
plt.show()

- stylecloud 패키지로 시각화

In [17]:
import stylecloud
stylecloud.gen_stylecloud(text = top_nouns_main,
                          icon_name="fas fa-broadcast-tower",
                          palette="colorbrewer.diverging.Spectral_11",
                          font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf',
                          background_color='black',
                          gradient="horizontal",
                          output_name="test.png")

### 기사 제목 


In [22]:
from konlpy.tag import Mecab

tagger = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

nouns_title= [ ]
for title in articles_title_clean:
  tagged = tagger.pos(title)
  nouns = [s for s, t in tagged if t in ['NNG', 'NNP', 'VA', 'XR'] and len(s) >1]
  for noun in nouns:
    if noun not in stop_words:
        nouns_title.append(noun)

In [ ]:

nouns_title_counter = Counter(nouns_title)
top_nouns_title = dict(nouns_title_counter.most_common(200))



In [ ]:
wc.generate_from_frequencies(top_nouns_title) 

figure =  plt.figure(figsize = (20, 20))
ax =  figure.add_subplot(1, 1, 1)
ax.axis('off')
ax.imshow(wc)
plt.show()

### 토픽 모델링 

- 토픽 모델링을 위해서 한 기사에서 나온 단어들끼리 리스트로 묶어서 [[],[]  .. ,[]] 형태로 만들어 준다.

In [12]:
tagger = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")


# 본문내용
nouns_article_lda = []
for article in articles_main_clean:
  tagged = tagger.pos(article)
  nouns = [s for s, t in tagged if t in ['NNG', 'NNP', 'VA', 'XR'] and len(s) >1]
  tmp_list = []
  for noun in nouns:
    if noun not in stop_words:
      tmp_list.append(noun)
      nouns_article_lda.append(tmp_list)



- gensim 패키지를 활용하여 토픽 모델링 진행

In [13]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

- Bag of words라고 하는 단어 가방을 만들기 위해 고유한 단어들의 사전을 만든다.

In [ ]:
dictionary = corpora.Dictionary(nouns_article_lda)

- 50%이상 나오는 단어, 5번 이하 나오는 단어는 필터링 

In [ ]:
dictionary.filter_extremes(no_below=5, no_above=0.5)

- 이렇게 사전이 만들어지면 사전 속의 단어가 문장에서 몇 번 출현하는지 빈도를 세서 벡터화. 

In [16]:
corpus = [dictionary.doc2bow(text) for text in nouns_article_lda]

- LDA
> 1. 토픽모델링을 몬서를 구성하는 몇 개의 토픽들이 존재하고 있으면 그 토픽들은 단어들의 집합으로 구성되어 있다고 가정  
  
> 2. 문서 내에 토픽들의 확률분포와 각 토픽을 구성하는 단어들의 확률분포가 주어졌을 때, 문서를 구성하는 토픽을 확률적으로 선택하고   
선택된 토픽에 존재하는 단어를 확률적으로 선택하는 샘플링 과정을 반복함으로써 임의의 문서를 생성하는 모델

In [ ]:
#10개 6개 5개 실행 

num_topics = 5
passes = 20
iterations = 400



ldamodel = LdaModel(corpus,
                    num_topics = num_topics,
                    id2word=dictionary, 
                    iterations= iterations,
                    alpha='auto',
                    eta='auto',
                    eval_every=None,
                    passes=passes)


- 모델 적용

In [ ]:
top_topics = ldamodel.top_topics(corpus) 

avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)


- Gensim의 토픽모델링 결과를 시각화하는 pyLDAvis 패키지 사용

In [25]:
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [ ]:
lda_visualization = gensimvis.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'articles_5.html')

In [27]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)
    

In [42]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

C:\Users\koreacamel\AppData\Local\Temp\ipykernel_3236\1543943919.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)


,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,3,0.4674,"[(0, 0.05251754), (1, 0.05077156), (2, 0.08843..."
1,1,3,0.4674,"[(0, 0.05251754), (1, 0.05077266), (2, 0.08843..."
2,2,3,0.4674,"[(0, 0.052517407), (1, 0.050768938), (2, 0.088..."
3,3,3,0.4674,"[(0, 0.052517407), (1, 0.050770912), (2, 0.088..."
4,4,3,0.4674,"[(0, 0.052517477), (1, 0.050769866), (2, 0.088..."
5,5,3,0.4674,"[(0, 0.052517485), (1, 0.050772354), (2, 0.088..."
6,6,3,0.4674,"[(0, 0.052517448), (1, 0.050771125), (2, 0.088..."
7,7,3,0.4674,"[(0, 0.05251733), (1, 0.05076939), (2, 0.08843..."
8,8,3,0.4674,"[(0, 0.052517418), (1, 0.050771352), (2, 0.088..."
9,9,3,0.4674,"[(0, 0.052517418), (1, 0.050770793), (2, 0.088..."


In [43]:
topictable.to_csv('topictable.csv', index=False)